In [1]:
import tensorflow as tf
from tensorflow.keras import layers

import keras_tuner

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv(r'../data/transformed_data.csv')
df_val = pd.read_csv(r'../data/transformed_2020_data.csv')

label_encoder = LabelEncoder()
label_encoder.fit(df['StateCode'])

X = df.drop(columns=['TotalAmountofAssistance']).values
y = df['TotalAmountofAssistance'].values
X[:, 0] = label_encoder.transform(X[:, 0])


X_val = df_val.drop(columns=['TotalAmountofAssistance']).values
y_val = df_val['TotalAmountofAssistance'].values
X_val[:, 0] = label_encoder.transform(X_val[:, 0])


X = X.astype('float32')
y = y.astype('float32')
X_val = X_val.astype('float32')
y_val = y_val.astype('float32')

In [3]:
def build_model(hp):
    model = tf.keras.Sequential()

    model.add(layers.Dense(hp.Choice('units', [8, 16, 32]), activation='sigmoid', input_shape=(X.shape[1],)))
    model.add(layers.Dense(hp.Choice('units', [8, 16, 32]), activation='relu'))
    model.add(layers.Dense(1, activation='linear'))


    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    loss = tf.keras.losses.MeanSquaredError()
    metrics = tf.keras.metrics.RootMeanSquaredError()

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=[metrics]
    )

    return model


build_model(keras_tuner.HyperParameters())

tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=5
)


INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [4]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Choice)
{'default': 8, 'conditions': [], 'values': [8, 16, 32], 'ordered': True}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [5]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X, y, epochs=5, validation_data=(X_val, y_val), callbacks=[stop_early])

KeyError: <keras_tuner.engine.trial.Trial object at 0x000001F1D5E4E310>

`